In [5]:
!git clone --depth 1 --branch master --no-checkout https://github.com/matheus-1618/supercomp.git
!cd supercomp&&git sparse-checkout set material/aulas/15-gpu-iteradores
!cd supercomp&&git checkout master
!mv supercomp/material/aulas/15-gpu-iteradores/* .
!rm -rf supercomp

Cloning into 'supercomp'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 240 (delta 22), reused 179 (delta 19), pack-reused 0
Receiving objects: 100% (240/240), 26.83 MiB | 35.45 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Already on 'master'
Your branch is up to date with 'origin/master'.


In [7]:
!./run.sh

1025202362 =  2*1804289383+1025202362
1350490027 =  2*846930886+1350490027
783368690 =  2*1681692777+783368690
1102520059 =  2*1714636915+1102520059
2044897763 =  2*1957747793+2044897763
1967513926 =  2*424238335+1967513926
1365180540 =  2*719885386+1365180540
1540383426 =  2*1649760492+1540383426
304089172 =  2*596516649+304089172
1303455736 =  2*1189641421+1303455736


In [15]:
%%writefile magnitude.cu
#include <iostream>
 #include <iomanip>
 #include <cstdlib>
 #include <chrono>
 #include <cstdlib>
 #include <algorithm>
 #include <thrust/host_vector.h>
 #include <thrust/device_vector.h>
 #include <thrust/generate.h>
 #include <thrust/functional.h>
 #include <thrust/copy.h>
 #include <cmath>
 #include <random>

 using namespace std::chrono;

 void reportTime(const char* msg, steady_clock::duration span) {
     auto ms = duration_cast<milliseconds>(span);
     std::cout << msg << " - levou - " <<
      ms.count() << " milisegundos" << std::endl;
 }

 // CRIE UMA FUNCTOR PARA CALCULAR A SQUARE
struct square {
    __host__ __device__
    float operator()(const float &x) const {
        return x * x;
    }
};


 // IMPLEMENTE O CALCULO DA MAGNITUDE COM THRUST
 float magnitude(thrust::device_vector<float> x) {
     float result;
     thrust::transform(x.begin() , x.end() , x.begin() , square());

     float sum_square = thrust::reduce(x.begin(), x.end());
     result = std::sqrt(sum_square);
     return result;
 }

 int main(int argc, char** argv) {
     if (argc != 2) {
         std::cerr << argv[0] << ": numero invalido de argumentos\n";
         std::cerr << "uso: " << argv[0] << "  tamanho do vetor\n";
         return 1;
     }
     int n = std::atoi(argv[1]); //numero de elementos
     steady_clock::time_point ts, te;

     // Faça um  vector em thrust
     thrust::host_vector<float> vetor(n);


     // inicilize o  vector
     ts = steady_clock::now();

     std::generate(vetor.begin(),vetor.end(), std::rand);


     te = steady_clock::now();
     reportTime("Inicializacao", te - ts);

     //Transferindo para gpu
     thrust::device_vector<float> v_d = vetor;

     // Calcule a magnitude do vetor
     ts = steady_clock::now();
     float len = magnitude(v_d);
     te = steady_clock::now();
     reportTime("Tempo para calculo", te - ts);


     std::cout << std::fixed << std::setprecision(4);
     std::cout << "Magnitude : " << len << std::endl;
 }

Overwriting magnitude.cu


In [21]:
!nvcc -arch=sm_70 magnitude.cu -o magnitude
!./magnitude 5

Inicializacao - levou - 0 milisegundos
Tempo para calculo - levou - 0 milisegundos
Magnitude : 3684236544.0000


In [17]:
%%writefile magnitude_fusion.cu
#include <iostream>
 #include <iomanip>
 #include <cstdlib>
 #include <chrono>
 #include <cstdlib>
 #include <algorithm>
 #include <thrust/host_vector.h>
 #include <thrust/device_vector.h>
 #include <thrust/generate.h>
 #include <thrust/functional.h>
 #include <thrust/copy.h>
 #include <cmath>
 #include <random>

 using namespace std::chrono;

 void reportTime(const char* msg, steady_clock::duration span) {
     auto ms = duration_cast<milliseconds>(span);
     std::cout << msg << " - levou - " <<
      ms.count() << " milisegundos" << std::endl;
 }

 // CRIE UMA FUNCTOR PARA CALCULAR A SQUARE
struct square {
    __host__ __device__
    float operator()(const float &x) const {
        return x * x;
    }
};


 // IMPLEMENTE O CALCULO DA MAGNITUDE COM THRUST
 float magnitude(thrust::device_vector<float> x) {
     float result;
     thrust::transform(x.begin() , x.end() , x.begin() , square());  // se não colocar nada ou só o begin ele sobrescreve o resultado no próprio vetor
      result = std::sqrt(thrust::transform_reduce(x.begin(), x.end(), square(), 0.0f, thrust::plus<float>()));
     return result;

 }

 int main(int argc, char** argv) {
     if (argc != 2) {
         std::cerr << argv[0] << ": numero invalido de argumentos\n";
         std::cerr << "uso: " << argv[0] << "  tamanho do vetor\n";
         return 1;
     }
     int n = std::atoi(argv[1]); //numero de elementos
     steady_clock::time_point ts, te;

     // Faça um  vector em thrust
     thrust::host_vector<float> vetor(n);


     // inicilize o  vector
     ts = steady_clock::now();

     std::generate(vetor.begin(),vetor.end(), std::rand);


     te = steady_clock::now();
     reportTime("Inicializacao", te - ts);

     //Transferindo para gpu
     thrust::device_vector<float> v_d = vetor;

     // Calcule a magnitude do vetor
     ts = steady_clock::now();
     float len = magnitude(v_d);
     te = steady_clock::now();
     reportTime("Tempo para calculo", te - ts);


     std::cout << std::fixed << std::setprecision(4);
     std::cout << "Magnitude : " << len << std::endl;
 }

Overwriting magnitude_fusion.cu


In [23]:
!nvcc -arch=sm_70 magnitude_fusion.cu -o magnitude_fusion
!./magnitude_fusion 5

Inicializacao - levou - 0 milisegundos
Tempo para calculo - levou - 0 milisegundos
Magnitude : 6514927451968110592.0000
